## Retrieving and analysing BRO soil cores 

In this example, we will download soil cores (BRO BHR-P objects) and calculate the total
thickness of sand, clay, silt and peat layers for each or the boreholes.

First we import GeoST and retrieve soil cores from the area of Landgoed Oostbroek
(located between Utrecht Science Park and Zeist). GeoST directly connects to the BRO
REST-API service. This allows the user to download BRO objects from a spatial 
query and apply GeoST functionality to the loaded objects. In this case we will use 
a bounding box:

In [ ]:
import geost

bhrp_cores = geost.bro_api_read("BHR-P", bbox=(141_000, 455_200, 142_500, 456_000)) # xmin, ymin, xmax, ymax

print(bhrp_cores)
print('\nThe header looks like this:')
bhrp_cores.header.gdf.head()

BRO objects come in the Rijksdriehoek coordinate reference system by default (EPSG: 28992),
but the example project requires us to work in UTM 31N coordinates (EPSG: 32631). Hence we 
convert the CRS and check the result:

In [ ]:
bhrp_cores.change_horizontal_reference(32631) # This is an in-place operation

print(bhrp_cores.horizontal_reference)
bhrp_cores.header.gdf.head()

Furthermore, we'd only like to work with boreholes that have a minimum length of 1.5 m

In [ ]:
bhrp_cores = bhrp_cores.select_by_length(1.5) # Not an in-place operation, so assign to new or overwrite existing variable!

print(bhrp_cores)

It looks like we ditched only two boreholes that were too short.

Next, we need to get the main lithology (Zand, Leem, Klei or Veen = Sand, Silt, Clay or Peat) 
of each described layer in our collection of boreholes. We can use the "standard_name" column
in the data, which is a lithological description including an indication of admixture:

In [ ]:
bhrp_cores.data["standardSoilName"].unique()

Since we are only interested in the main lithology we need to simplify the names. There 
is no method for this provided by GeoST, so we will have to create our own. Luckily, GeoST
is built on Pandas, which means that you can easily apply your own logic if
built-in GeoST methods don't get you where you want. In this case, we will use a string method of a Pandas Series to use a regex (i.e. regular expression) which extracts the main lithology. This will reduce e.g. "matigSiltigeKlei" to "Klei".

In [ ]:
bhrp_cores.data["lithology"] = bhrp_cores.data["standardSoilName"].str.extract(
    r'(Klei|Zand|Leem|Veen)$', expand=False # Find one of these words at the end of the standardSoilName
)

# Check and show that it worked:
bhrp_cores.data["lithology"].unique()

Now we can proceed to compute the total thickness of our main lithologies by applying
the built-in method "get_cumulative_thickness" on our just created "main_lithology" column.

In [ ]:
# By including the "include_in_header=True" argument, the result will be added in-place
# to the header table of the BoreholeCollection. If False, the function will only return
# a Pandas dataframe with the results.
bhrp_cores.get_cumulative_thickness(
    "lithology",
    ["Klei", "Zand", "Leem", "Veen"],
    include_in_header=True
)

# Check if the new columns are present and correct in the header table
bhrp_cores.header.gdf.head()

For the purpose of this example we will quickly plot the results on a map below. Hover your mouse
over the points to display header information, including the layer thicknesses that we just added.
From this point you would normally export the data or continue with further data analyses.

In [ ]:
bhrp_cores.header.gdf.explore(style_kwds=dict(color="red", weight=6))